<a href="https://colab.research.google.com/github/VishalRavichandran27/hds5210-2023/blob/main/week15/week15_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 15 Programming Assignment

The final thing for this semester that we haven't exercised, yet, is working with databases through Python.  In thi final assignment of the semester, you will practice pulling data from either Google Big Query or Snowflake, loading that data into a Pandas data frame, summarizing the data, and then exporting that to an Excel file.

**You will need to use your own Google Big Query or Snowflake account to run this notebook, but you should try to make it configurable so anyone with an account in those technologies could run your notebook with minimal changes.**


Build a notebook with good comments (either in code or in markdown cells).  Then submit your assignment as usual.

### 1. Connect to the data source

Follow the examples provided in the [week15 folder of our GitHub repository](https://github.com/paulboal/hds5210-2023/tree/main/week15) to connect your notebook either to Big Query or Snowflake.

In [5]:
# Installing the packages required
!pip install "snowflake-connector-python[pandas]"

In [6]:
import snowflake.connector
# I have created a dictionary to put all my snowflake detailes as details
details = {"snowflake_username" : 'VISHAL',
           "snowflake_password" : 'Vishal@2710',
           "snowflake_account" : 'DBAINHO-PL25312',
           "snowflake_warehouse" : 'COMPUTE_WH',
           "snowflake_database" : 'ONCOLOGY_DEMO',
           "snowflake_schema" : 'OMOP',
           "snowflake_query" : 'SELECT * FROM COST;'
           }
           # My SQL Query


### 2. Query some data

Assuming the source database has some data in it or that you can load some data into it from any source, query it.  Then, read that data into a Pandas dataframe.

In [7]:
# Here i am calling the details from the dictionary as Sf_con
with snowflake.connector.connect(
        user=details["snowflake_username"],
        password=details["snowflake_password"],
        account=details["snowflake_account"],
        warehouse=details["snowflake_warehouse"],
        database=details["snowflake_database"],
        schema=details["snowflake_schema"]
) as Sf_con:

  # Creating a cursor object as Sf_cursor
  Sf_cursor = Sf_con.cursor()

  # Executing the above snowflake query
  Sf_cursor.execute(details["snowflake_query"])

  # Here i used the below code which will fetch all the results into a Pandas DataFrame
  data = Sf_cursor.fetch_pandas_all()
  data

### 3. Aggregate your data frame

Do some kind of aggregation on your data frame.  Something that makes sense and has some groups to it.  Don't just sum up one column for the entire data frame.  Be more creative than that.

In [8]:
data

,PERSON_ID,COST_ID,COST_DATE,PAYER_CATEGORY,TOTAL_CHARGE,TOTAL_PAID
0,67131674,b5d41875-b1c9-40e6-b5f6-3a95b4c0150f,2016-09-09,Medicare/Medicaid,230.0,NaN
1,67131674,505c47c2-b61a-43cd-9144-ecfd9e5428be,2016-05-13,Medicare/Medicaid,2.0,NaN
2,67131674,54ee5376-7cd1-4742-80f8-25e740a0c5e6,2016-09-23,Medicare/Medicaid,12.0,NaN
3,67131674,8488b01b-fd2f-4a3c-942c-6a94e296fe76,2020-04-20,,NaN,60.0
4,67131674,4ff94326-fded-42cd-b286-494209dae236,2016-09-09,,NaN,12.0
...,...,...,...,...,...,...
773529,75568761,20a664a5-546f-4f5d-8a25-1dd544c8f76c,2018-04-13,,NaN,6.0
773530,75568761,8c0b4d78-a23b-4cb2-ad75-f6f6aa5302d5,2018-04-13,,NaN,0.0
773531,75568761,8c0b4d78-a23b-4cb2-ad75-f6f6aa5302d5,2018-04-13,Other,0.0,NaN
773532,75568761,2d2b1f0c-4540-4a18-82f9-7dec85260d1a,2018-04-13,Other,27.0,NaN


In [9]:
# Here in i am using only the 3 columns to check the cost
data = data[['COST_DATE','TOTAL_CHARGE','TOTAL_PAID']]
data

,COST_DATE,TOTAL_CHARGE,TOTAL_PAID
0,2016-09-09,230.0,NaN
1,2016-05-13,2.0,NaN
2,2016-09-23,12.0,NaN
3,2020-04-20,NaN,60.0
4,2016-09-09,NaN,12.0
...,...,...,...
773529,2018-04-13,NaN,6.0
773530,2018-04-13,NaN,0.0
773531,2018-04-13,0.0,NaN
773532,2018-04-13,27.0,NaN


In [10]:
# Here i am checking the data type where the cost date column was object
data.dtypes

COST_DATE        object
TOTAL_CHARGE    float64
TOTAL_PAID      float64
dtype: object

In [11]:
# Here i have changed the cost date datatype into datetime format
import pandas as pd
data['COST_DATE'] = pd.to_datetime(data['COST_DATE'])
data.dtypes

<ipython-input-11-663eb60e0537>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['COST_DATE'] = pd.to_datetime(data['COST_DATE'])


COST_DATE       datetime64[ns]
TOTAL_CHARGE           float64
TOTAL_PAID             float64
dtype: object

In [12]:
# I have grouped by and gave a function to sum up all the cost and sorted it to get in a proper format
data = data.groupby(by = 'COST_DATE').agg('sum').sort_index().reset_index()

### 4. Write to Excel

Use Pandas functions to write your summarized data out to a local Excel file.

In [13]:
# Here i have converted the data into the excel file as per the requirements
data.to_excel('ONCO.xlsx')

---

## Submitting Your Work

Submit your work as usual